# rappel structure des données:  
Grey images are stored in memory as 2D arrays, color images as a 3D arrays

io.imread returns a np.array of shape (hauteur, largeur, 3) or (hauteur, largeur)
* image [ 0 , 0 ] : upper left corner
* image [ 0 , n ] : upper right corner
* image [ n , 0 ] : lower left corner
* image [ : , 0:n/2 ] : left half of picture

size of video picture: 1080, 1920

doc: https://docs.opencv.org/master, mostly in the features2D and calib3d modules

In [ ]:
%matplotlib widget
%matplotlib inline

import numpy as np
import math
import cv2 as cv2 # opencv computer vision library
from skimage import io # for io.imread
from matplotlib import pyplot as plt # ploting
from matplotlib import colors # ploting

import mediapipe as mp  # human pose detection

import ast # string representation of list to list using ast.literal_eval()

# interactive notebook widgets
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [ ]:
root_directory = 'data/'

plan_large_1 = io.imread(root_directory + '01.18.42.183_front.png')
plan_cote_1 =  io.imread(root_directory + '01.18.47.450_side1.png')
plan_cote_2 =  io.imread(root_directory + '01.18.52.000_side2.png')

travel_front = io.imread_collection (root_directory + 'travel_front/*')

# autre
plan_face_autre = io.imread(root_directory + 'other/front.jpg')
plan_cote_autre = io.imread(root_directory + 'other/side.jpg')
plan_shema = io.imread(root_directory + 'other/shematic.jpg')

In [ ]:
def imshow(images, titles="123456789", callback = None, nrows = 0, ncols=0, figsize = (15,20)):
    """Plot a multiple images with titles.

    Parameters
    ----------
    images : image list
    titles : title list
    ncols : number of columns of subplots wanted in the display
    nrows : number of rows of subplots wanted in the figure
    """

    if ncols == 0 and nrows == 0:
        ncols = len(images)
        nrows = 1
    if ncols == 0:
        ncols = len(images) // nrows
    if nrows == 0:
        nrows = len(images) // ncols

    fig, axeslist = plt.subplots(ncols=ncols, nrows=nrows, squeeze=False, figsize = figsize)
    for i, image in enumerate(images):
        axeslist.ravel()[i].imshow(image, cmap=plt.gray(), vmin=0, vmax=255)
        axeslist.ravel()[i].set_title(titles[i])
        axeslist.ravel()[i].set_axis_off()
    plt.tight_layout() # optional

    if callback is not None:
        def onclick(event):
            [i],[j] = np.where(axeslist == event.inaxes)
            callback(axeslist, [i,j], [event.xdata, event.ydata])

        # Create an hard reference to the callback not to be cleared by the garbage collector
        ka = fig.canvas.mpl_connect('button_press_event', onclick)
    return axeslist

print (len(travel_front))
axeslist = imshow (travel_front)
print(type(axeslist))
print(axeslist.shape)

In [ ]:
def extract_key_points(img1, img2):
    akaze = cv2.AKAZE_create()
    kp1, des1 = akaze.detectAndCompute (img1,None)
    kp2, des2 = akaze.detectAndCompute (img2,None)
    return kp1, des1, kp2, des2

def showKeyPoints(img1, kp1, img2, kp2):
    img_1 = cv2.drawKeypoints( img1, kp1, None) 
    img_2 = cv2.drawKeypoints( img2, kp2, None) 
    imshow([img_1, img_2],["left", "right"])

def match_key_points(kp1, des1, kp2, des2):
    bf = cv2.BFMatcher()
    # Match descriptors.
    matches = bf.match(des1,des2)
    # Sort them in the order of their distance.
    return sorted(matches, key = lambda x:x.distance)

def showMatches(img1, kp1, img2, kp2, matches, name):
    img = cv2.drawMatches(img1,kp1,img2,kp2,matches,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    imshow([img],[name])

def findHomography(matches, keypoint1, keypoint2):
    src_pts = np.array([keypoint1[matches[m].queryIdx].pt for m in range(len(matches))])
    dst_pts = np.array([keypoint2[match.trainIdx].pt for match in matches])
    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    inliers = [matches[m] for m in range(len(mask)) if mask[m]==1]
    return H,inliers

In [ ]:
def panorama_pipeline (images,final_size, location="ul"):
    x,y, = final_size
    nb_img = len(images)
    img_pano = images[0]

    y1,x1, _ = images[0].shape
    H = np.array([[1.,0.,0.],[0.,1.,0.],[0.,0.,1.]])
    # location = where to place first image
        # first letter = up/middle/bottom
        # second letter = left/middle/right
    if location[1]=='m':
        H[0][2] = (x1-x)/2
    if location[1]=='r':
        H[0][2] = x1-x
    if location[0]=='m':
        H[1][2] = (y1-y)/2
    if location[0]=='b':
        H[1][2] = y1-y

    img_pano =  255*np.ones((y, x, 3),dtype=np.uint8)
    img_pano = cv2.warpPerspective(images[0], H, final_size, img_pano, borderMode=cv2.BORDER_TRANSPARENT, flags=cv2.WARP_INVERSE_MAP)

    for i in range (1,nb_img):
        img1= images[i-1]
        img2= images[i]
        kp_1, des_1, kp_2, des_2 = extract_key_points(img1, img2)
        matches_pipeline = match_key_points(kp_1, des_1, kp_2, des_2)
        H2, inliers = findHomography(matches_pipeline, kp_1, kp_2)
        H=np.matmul(H2, H)
        img_pano = cv2.warpPerspective(img2, H, final_size, img_pano, borderMode=cv2.BORDER_TRANSPARENT, flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
    return img_pano

pano = panorama_pipeline(travel_front,(3000,4000), "bm")
imshow([pano], ["panorama"] )


### Extracting edges by hand (clicking points)

In [ ]:
def draw_point(axe, point, color):
    # draw the clicked point
    axe.scatter([point[0]], [point[1]], color=[color], s=400, marker='+')

def onclick_get_coord(axeslist, ij, p):
    i = ij[0] # row == 0
    j = ij[1] # column == 0 or 1 for left or right image
    color = np.random.rand(3) # take a random color 
    draw_point(axeslist[i, j], p, color)
    print(f"{p[0]}  {p[1]}")

# axeslist = imshow( [plan_cote_1, plan_large_1], ['côté', 'face'], onclick_get_coord)
axeslist = imshow( [plan_face_autre], ['autre'], onclick_get_coord)

In [ ]:
"""f = open("data/front_other_face.txt", "r")
L_front_other = []
for line in f:
  coords = ast.literal_eval(line)
  L_front_other.append(coords)"""

In [ ]:
color = np.random.rand((3))
counter = 0


def onclick_get_all_coord(axeslist, ij, p):
    global color, counter
    i = ij[0] # row == 0
    j = ij[1] # column == 0 or 1 for left or right image
    # take a random color
    if i==0:
        color = np.random.rand((3))
    draw_point(axeslist[i, j], p, color)
    print(f"{counter}  {i}  {p[0]}  {p[1]}")
    if i==axeslist.size-1:
        counter += 1

# axeslist = imshow( [plan_cote_1, plan_large_1], ['côté', 'face'], onclick_get_coord)
axeslist = imshow( [plan_face_autre, plan_cote_autre], ['face', 'côté'], onclick_get_all_coord, ncols=1)

In [ ]:
color = np.random.rand((3))
counter = 0

def onclick_show_place(axeslist, ij, p):
    global color, counter, L_front_other
    i = ij[0] # row == 0
    j = ij[1] # column == 0 or 1 for left or right image
    # take a random color
    if i==0:
        color = np.random.rand((3))
        draw_point(axeslist[i, j], L_front_other[counter], color)
        counter += 1
    else:
        draw_point(axeslist[i, j], p, color)
        print(f"{counter}:   {p[0]}  {p[1]}")

# axeslist = imshow( [plan_cote_1, plan_large_1], ['côté', 'face'], onclick_get_coord)
axeslist = imshow( [plan_face_autre, plan_cote_autre], ['face', 'côté', 'shema'], onclick_show_place, ncols=1)

### trying a line detector to detect facettes

In [ ]:
dst = cv2.Canny(plan_cote_1, 0, 50, None, 3)

# Copy edges to the images that will display the results in BGR
cdst = cv2.cvtColor(dst, cv2.COLOR_GRAY2BGR)
cdstP = np.copy(cdst)

lines = cv2.HoughLines(dst, 1, np.pi / 180, 150, None, 0, 0)

if lines is not None:
    for i in range(0, len(lines)):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        a = math.cos(theta)
        b = math.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
        pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
        cv2.line(cdst, pt1, pt2, (0,0,255), 3, cv2.LINE_AA)


linesP = cv2.HoughLinesP(dst, 1, np.pi / 180, 50, None, 50, 10)

if linesP is not None:
    for i in range(0, len(linesP)):
        l = linesP[i][0]
        cv2.line(cdstP, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv2.LINE_AA)

imshow([plan_cote_1, dst, cdst, cdstP], ["source", "canny", "standard detection", "probabilistic detection"], nrows=2)

### trying color blob detector to identify facettes
does not work because blobs to  big ?

In [ ]:
# Set up the detector with default parameters.
detector = cv2.SimpleBlobDetector_create()
 # Detect blobs.
keypoints = detector.detect(plan_cote_1)
img_with_kp = cv2.drawKeypoints( plan_cote_1, keypoints, None)
imshow([img_with_kp], ["image with kpts"])

# detecting human pose with media pipe

### time stamp of climbers:
1h24: first man

In [ ]:
# initialize Pose estimator
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

pose = mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

In [ ]:
# create capture object
cap = cv2.VideoCapture('data/video.mp4')

print(type(cap))

while cap.isOpened():
    print("ok")
    # read frame from capture object
    _, frame = cap.read()

    # convert the frame to RGB format
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # process the RGB frame to get the result
    results = pose.process(rgb_frame)
    print(results.pose_landmarks)
    # draw detected skeleton on the frame
    mp_drawing.draw_landmarks(
    	frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # show the final output
    cv2.imshow('Output', frame)

    if cv2.waitKey(1) == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()